ANIMACIÓN: DIVERGENCIA + VECTORES DE VIENTO (200hPa)

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import pandas as pd

# Carga del dataset
ds = xr.open_dataset("ERA_2025_final3.nc")
# Periodo de estudio (19 al 30 de junio con frecuencia de 6 horas)
fechas = pd.date_range("2025-06-19 00:00", "2025-06-30 18:00", freq="6H")
# Coordenadas de toda sudamérica
extent = [-120, 10, -70, 20]
# Variable del nivel de presión
nivel = 200

# Se verifica que exista la versión expver, necesario para data con vacios
if 'expver' in ds.dims:
    ds = ds.sel(expver=1).combine_first(ds.sel(expver=5))

# Figura base de la carta sinóptica, se define el tamaño y la proyección
fig, ax = plt.subplots(figsize=(11, 8), subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_extent(extent, crs=ccrs.PlateCarree())
# Delimitacín continete-océano/fronteras geográficas entre países, esto permite ubica la región de estudio (Atacama-Chile)
ax.add_feature(cfeature.COASTLINE, linewidth=1)
ax.add_feature(cfeature.BORDERS, linewidth=0.6)
# Grillas
gl = ax.gridlines(draw_labels=True, linewidth=0.3, linestyle='--')
gl.top_labels = gl.right_labels = False

# Barra de colores 
niveles_v = np.arange(20, 80, 5) # se delimita el maximo de velocidad>20 m/s con un intervalo de 5m/s
dummy = ax.contourf(
    [-120, -119], [-70, -69],
    [[20, 20], [20, 20]],
    levels=niveles_v, cmap='BuPu', extend='max',
    transform=ccrs.PlateCarree()
)
cbar = plt.colorbar(dummy, ax=ax, pad=0.02, shrink=0.7)
cbar.set_label("Velocidad del viento (m/s)")

# FUNCIÓN 
def actualizar(i):
    ax.clear() # Limpieza y selección temporal
    ax.set_extent(extent, crs=ccrs.PlateCarree())
    # Selecciona el tiempo más cercano ("nearest") y extrae solo el nivel de 200hPa
    fecha = fechas[i]
    d = ds.sel(pressure_level=nivel, valid_time=fecha, method='nearest')
    # Variables meteorológicas (viento zonal, meridional y geopotencial)
    u = d.u
    v = d.v
    z = d.z / 9.81
    speed = np.sqrt(u**2 + v**2) #Calculo de la velocidad
    # Máscara del jet >=20 m/s (esto elimina los vientos débiles en altura)
    speed_mask = speed.where(speed >= 20)
    speed_cyc, lon_cyc = add_cyclic_point(speed_mask.values, coord=d.longitude)
    z_cyc, _ = add_cyclic_point(z.values, coord=d.longitude)
    #Sombreado de los jets
    im = ax.contourf(
        lon_cyc, d.latitude, speed_cyc,
        levels=niveles_v, cmap='BuPu', extend='max',
        transform=ccrs.PlateCarree()
    )
    # Contorno de altura geopotencial (permite identificar vaguadas y dorsales)
    niveles_z = np.arange(10500, 12500, 120)
    cs = ax.contour(
        lon_cyc, d.latitude, z_cyc,
        levels=niveles_z, colors='black', linewidths=1.2,
        transform=ccrs.PlateCarree()
    )
    ax.clabel(cs, fmt='%d', fontsize=10)
    # Vectores de viento, el salto evita densidad visual de vectores
    salto = 20
    v_max = speed.max().values
    ax.quiver(
        d.longitude[::salto], d.latitude[::salto],
        u[::salto, ::salto], v[::salto, ::salto],
        scale=v_max*12, width=0.0025, color='black',
        transform=ccrs.PlateCarree()
    )

    ax.add_feature(cfeature.COASTLINE, linewidth=1)
    ax.add_feature(cfeature.BORDERS, linewidth=0.6)
    gl = ax.gridlines(draw_labels=True, linewidth=0.3, linestyle='--')
    gl.top_labels = gl.right_labels = False
    # Título dinámico
    ax.set_title(
        f"Corrientes en chorro y Altura Geopotencial\n{fecha.strftime('%Y-%m-%d %H:%M')} UTC",
        fontsize=12, fontweight='bold'
    )

    return im


# Animación
ani = FuncAnimation(fig, actualizar, frames=len(fechas), interval=300)

plt.close(fig)
HTML(ani.to_jshtml())


ANIMACIÓN: DIVERGENCIA EN 200HPA

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import pandas as pd

# Carga de dataset
dg = xr.open_dataset("ERA_2025_final.nc")
# Periodo de estudio (19 al 30 de junio con frecuencia de 6 horas)
fechas = pd.date_range("2025-06-20 00:00", "2025-06-30 18:00", freq="6H")
# Coordenadas de toda sudamérica
extent = [-120, 10, -60, 20]
# Variable del nivel de presión
nivel = 200

# Se verifica que exista la versión expver, necesario para data con vacios
if 'expver' in dg.dims:
    dg = dg.sel(expver=1).combine_first(dg.sel(expver=5))

# Figura base de la carta sinóptica, se define el tamaño y la proyección
fig, ax = plt.subplots(figsize=(12, 7), subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_extent(extent, crs=ccrs.PlateCarree())

#Se delimita el máximo y mínimo
levels_div = np.linspace(-6, 6, 21)

# Barra de colores fijo
dummy = ax.contourf(
    [-100, -99], [-60, -59],
    [[-6, -6], [-6, -6]],
    levels=levels_div, cmap="PuOr_r", extend="both",
    transform=ccrs.PlateCarree()
)

cbar = plt.colorbar(dummy, ax=ax, pad=0.02, shrink=0.8)
cbar.set_label("Divergencia (×10⁻⁵ s⁻¹)", fontsize=10)

# FUNCIÓN

def actualizar(i):
    ax.clear()# Limpieza y selección temporal
    ax.set_extent(extent, crs=ccrs.PlateCarree())
    ## Selecciona el tiempo más cercano ("nearest") y extrae solo el nivel de 200hPa
    fecha = fechas[i]
    ds = dg.sel(pressure_level=nivel, valid_time=fecha, method='nearest')
    # Calculo de divergencia
    div = ds.d * 1e5  # Escala de 10^5 
    z = ds.z / 9.81  # Geopotencial (z)

    div_cyc, lon_cyc = add_cyclic_point(div.values, coord=ds.longitude)
    z_cyc, _ = add_cyclic_point(z.values, coord=ds.longitude)

    # Sombreado de divergencia
    im = ax.contourf(
        lon_cyc, ds.latitude, div_cyc,
        levels=levels_div, cmap="PuOr_r",
        extend="both", transform=ccrs.PlateCarree(), alpha=0.85
    )

    # Contornos de geopotencial
    niveles_z = np.arange(10800, 12400, 120)

    cs = ax.contour(
        lon_cyc, ds.latitude, z_cyc,
        levels=niveles_z, colors="black", linewidths=1.2,
        transform=ccrs.PlateCarree()
    )
    ax.clabel(cs, fmt="%d", fontsize=9)

    # Delimitacín continete-océano/fronteras geográficas entre países, esto permite ubica la región de estudio (Atacama-Chile)
    ax.add_feature(cfeature.COASTLINE, linewidth=1.2, zorder=5)
    ax.add_feature(cfeature.BORDERS, linewidth=0.6, zorder=5)
    # Grillas
    gl = ax.gridlines(draw_labels=True, linewidth=0.4, color='gray', linestyle='--')
    gl.top_labels = gl.right_labels = False
    #Título dinámico
    ax.set_title(
        f"Campo de Divergencia en Altura (200 hPa)\n{fecha.strftime('%Y-%m-%d %H:%M')} UTC",
        fontsize=13, fontweight='bold'
    )

    return im

# Animación
ani = FuncAnimation(fig, actualizar, frames=len(fechas), interval=300)

plt.close(fig)
HTML(ani.to_jshtml())
